# Agen AI Azure dengan Dukungan Model Context Protocol (MCP)

Notebook ini menunjukkan cara menggunakan Agen AI Azure dengan alat Model Context Protocol (MCP) untuk membuat agen cerdas yang dapat memanfaatkan server MCP eksternal untuk kemampuan yang lebih baik.


## Instal Paket NuGet yang Diperlukan

Pertama, kita perlu menginstal paket Azure AI Agents Persistent yang menyediakan fungsi inti untuk bekerja dengan Azure AI Agents.


## Manfaat Autentikasi Tanpa Kunci

Notebook ini menunjukkan **autentikasi tanpa kunci** yang memberikan beberapa keuntungan:
- ✅ **Tidak ada API key yang perlu dikelola** - Menggunakan autentikasi berbasis identitas Azure
- ✅ **Keamanan yang lebih baik** - Tidak ada rahasia yang disimpan dalam kode atau konfigurasi
- ✅ **Rotasi kredensial otomatis** - Azure menangani siklus hidup kredensial
- ✅ **Akses berbasis peran** - Menggunakan Azure RBAC untuk izin yang lebih terperinci

`DefaultAzureCredential` secara otomatis akan menggunakan sumber kredensial terbaik yang tersedia:
1. Managed Identity (saat berjalan di Azure)
2. Kredensial Azure CLI (selama pengembangan)
3. Kredensial Visual Studio
4. Variabel lingkungan (jika dikonfigurasi)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Instal paket Azure Identity untuk autentikasi dengan layanan Azure menggunakan DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Impor Namespace yang Diperlukan

Impor namespace yang diperlukan untuk Azure AI Agents dan Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Konfigurasi Azure AI Agent Client (Autentikasi Tanpa Kunci)

Atur variabel konfigurasi dan buat PersistentAgentsClient menggunakan **autentikasi tanpa kunci**:
- **projectEndpoint**: Endpoint proyek Azure AI Foundry
- **modelDeploymentName**: Nama model AI yang telah diterapkan (GPT-4.1 nano)
- **mcpServerUrl**: URL server MCP (Microsoft Learn API)
- **mcpServerLabel**: Label untuk mengidentifikasi server MCP
- **DefaultAzureCredential**: Menggunakan identitas terkelola, Azure CLI, atau sumber kredensial lainnya (tidak memerlukan kunci API)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Membuat Definisi Alat MCP

Buat definisi alat MCP yang terhubung ke server Microsoft Learn MCP. Ini akan memungkinkan agen untuk mengakses konten dan dokumentasi Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Membuat Agen AI

Buat agen AI yang persisten dengan model dan alat MCP yang telah ditentukan. Agen ini dikonfigurasi dengan:
- Model GPT-4.1 nano
- Instruksi untuk menggunakan alat MCP sebagai bantuan
- Akses ke server Microsoft Learn MCP


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Membuat Thread dan Mengirim Pesan

Buat sebuah thread percakapan dan kirim pesan kepada pengguna yang menanyakan perbedaan antara Azure OpenAI dan OpenAI. Ini akan menguji kemampuan agen untuk menggunakan alat MCP guna memberikan informasi yang akurat.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Konfigurasi Sumber Daya Alat MCP (Tanpa Kunci)

Atur sumber daya alat MCP. Untuk pendekatan yang benar-benar tanpa kunci, Anda dapat menghapus header khusus jika server MCP mendukung autentikasi berbasis identitas Azure. Contoh di bawah ini menunjukkan cara menambahkan header jika diperlukan, tetapi untuk skenario tanpa kunci, ini mungkin tidak diperlukan.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Mulai Menjalankan Agen

Buat dan mulai menjalankan untuk memproses pesan pengguna. Agen akan menggunakan alat dan sumber daya MCP yang telah dikonfigurasi untuk menghasilkan respons.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Memantau Jalannya Agen dan Menangani Persetujuan Alat (Tanpa Kunci)

Pantau status jalannya agen dan tangani persetujuan alat yang diperlukan. Proses ini:
1. Menunggu hingga jalannya selesai atau membutuhkan tindakan
2. Secara otomatis menyetujui panggilan alat MCP saat diperlukan
3. Untuk autentikasi tanpa kunci, header mungkin tidak diperlukan jika server MCP mendukung identitas Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Menampilkan Hasil Percakapan

Ambil dan tampilkan semua pesan dalam percakapan, menunjukkan pertanyaan pengguna dan respons agen. Pesan-pesan ditampilkan dalam urutan kronologis dengan penanda waktu dan indikator peran.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diketahui bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
